**Datenerhebung**

Die Daten wurden von Spotify erhoben, die diverse Analysedaten zu einer großen Anzahl von Liedern zur Verfügung stellen. Mithilfe einer HTTP-GET kann eine Anfrage an die Server von Spotify geschickt werden und man erhält als Antwort diverse Daten zu den Liedern. Ziel ist einen Datensatz zu erheben mit Liedern, die von Spotify zu einem bestimmten Genre zugeordnet werden. Zusätzlich werden zu jedem Lied, weitere Analyse Daten erhoben. Dadurch soll ein Modell entwickelt werden, welches unbekannten Lieder automatisch zu einem Genre zuordnen kann. 

Zuerst werden die beiden Module *Pandas* und *Requests* geladen. Pandas dient zur Erstellung und Bearbeitung von Tabellen. Das Requests Modul wird für die HTTP-Anfragen genutzt.

In [ ]:
import requests
import pandas as pd

Das *header*-Objekt muss immer im header bei den HTTP-Anfragen mitübergeben werden und beinhaltet eine Authentifizierung Token. Der Token wird von Spotify generiert und ist für eine Stunde gültig, bis ein neuer von Spotify generiert werden muss. 

In [ ]:
token = "Bearer BQC78LmLIZDkvkJzjqpsBdGArNlfCxICNMfHtG9EX1TiVgR3LfZ0wrwbbPnVYWCRbL0EMhSfqGGQQTEtqs3pgxuy1GpB-J1B2JhWUvY1UHYFfF5cILmOqEUVBvfy1guVmj6uvkj5piGJBrF3vUzqH1k3Qn34StO9u8z3PWrwFby_M6Y"
header = {
    "Authorization": token
}

Die Spotify API bietet die Möglichkeit, sich Informationen über mehrere Playlisten zu beschaffen, die in einer bestimmten Kategorie fallen. Die Liste *genre* beinhaltet die IDs von den Kategorien, wodurch sich zum Beispiel Informationen abrufen lassen über alle Playlisten, die von Spotify erstellt, die in die Kategorie Pop fallen. Es fällt auf, dass die ID für das Metal Genre anders aufgebaut ist als die anderen IDs der Genres und anstatt aus dem Wort „metal“, aus einer Kombination aus Zahlen und Buchstaben besteht.

In [ ]:
genre = ["pop","hiphop","edm_dance","rock","party","indie_alt","alternative","0JQ5DAqbMKFDkd668ypn6O"]

Über eine HTTP-GET anfrage lassen sich weitere Informationen zu den einzelnen Kategorien von Spotify beschaffen [[Kategorien](https://developer.spotify.com/console/get-browse-categories/), Abrufdatum: 11.07.2022]


In [ ]:
%%time
rows = []
for i in genre:
    genr = i
    
    get_playlistID = f"https://api.spotify.com/v1/browse/categories/{genr}/playlists"
    playlistID = requests.get(get_playlistID,headers=header)
    if playlistID.status_code == 200:
        playlistID = playlistID.json()
        for j in playlistID["playlists"]["items"]:
            rows.append({"id":j["id"],"name":j["name"],"gerne":i})

Die gesammelten Playlisten lassen sich in einen Datenframe umformen und durch, das Gruppieren nach Genre, lässt sich erkennen, dass zu jedem Genre nahezu gleichviele Playlisten gibt. 


In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df.groupby("gerne").size()

In [ ]:
#Umbennen des Genres für Playlisten in der Kategorie Metal
df.loc[df["gerne"] == "0JQ5DAqbMKFDkd668ypn6O","gerne"] = "metal"
df.groupby("gerne").size()

In [ ]:
rows = []
#Parameter für die Abfrage, da nur der Titel eines Liedes und die ID von interesse ist
params = {
    "fields" : "items(track(name,id))"
}

Mithelfe der gesammelten Informationen über die Playlisten, lassen sich mit einer weiteren HTTP-GET anfrage Informationen über den Inhalt der Playlisten beschaffen. Die Titel und IDs der Lieder werden wieder in einer Liste zwischengespeichert. Da die Lieder sich immer in einer Playlist zu einem bestimmten Genre befanden, können so die einzelnen Lieder einem Genre zugewiesen werden. 

In [ ]:
%%time
for i in range(0,len(df)):
    play_ID = df["id"][i]
    url = f"https://api.spotify.com/v1/playlists/{play_ID}/tracks"
    
    #fetchdata
    x = requests.get(url,params=params,headers=header)
    if x.status_code != 200:
        print(f"Fehlercode: {x.status_code}")
        break
    x_dict = x.json()
    
    #Save Data
    for j in x_dict["items"]:
        try:
            rows.append({"id":j["track"]["id"],"titel":j["track"]["name"],"genre":df["gerne"][i]})
        except TypeError:
            pass

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

Lieder können sich in mehreren Playlisten befinden, auch zu anderen Genres, können die Lieder nicht eindeutig einem Genre zugewiesen werden. Um die Duplikate zu entfernen und die Lieder eindeutig einem Genre zuzuweisen, wird die Tabelle nach den IDs gruppiert und nach der Häufigkeit des Auftretens in den Genres sortiert. Ein Lied das zum Beispiel in drei Playlisten zum Genre Metal zu finden war und in zwei Playlisten zum Genre Rock, wird so dem Genre Metal zugewiesen.

In [ ]:
df = df.groupby("id")["titel","genre"].max().reset_index()
df.shape

In [ ]:
df.head()

Durch die IDs der einzelnen Lieder lassen sich analytische Informationen zu jedem Lied beschaffen, die von Spotify zur Verfügung gestellt werden. Zu den Informationen gehören unteranderem die Länge, die Tonart, das Tempo und der Anteil an gesprochenen Worten eines Liedes. 


In [ ]:
%%time
track_id = df["id"]
track_featured_list = []

for i in track_id:
    url_track_feature = f"https://api.spotify.com/v1/audio-features/{i}"
    get_track_features = requests.get(url_track_feature,headers=header)
    
    if get_track_features.status_code != 200:
        print(f"Fehlercode: {get_track_features.status_code}")
        continue
    
    track_features = get_track_features.json()
    
    track_features.pop("type",None)
    track_features.pop("uri",None)
    track_features.pop("track_href",None)
    track_features.pop("analysis_url",None)
    
    track_featured_list.append(track_features)

Die Informationen werden erneut in einer Liste zwischengespeichert und anschließend in einer Tabelle umgeformt.

In [ ]:
df_temp = pd.DataFrame(track_featured_list)

In [ ]:
df_temp.head()

Die Tabelle mit den Liedern zu einem Genre und die Tabelle mit den analytischen Daten zu den Liedern, lassen sich zu einer Tabelle anhand der ID zusammenführen und kann anschließend als CSV-Datei exportiert werden.

In [ ]:
df_dump = df.merge(df_temp,on="id",how="inner")
print(df.shape)
print(df_temp.shape)
print(df_dump.shape)

In [ ]:
df_dump = df_dump.drop_duplicates(subset=["id","genre"],ignore_index=True)
df_dump.shape

In [ ]:
df_dump.to_csv("music_data.csv",index=False)